In [40]:
attendance = pd.read_csv("datas/attendance.csv")
climate = pd.read_csv("datas/preprocessing_jamsil.csv")
kbo = pd.read_csv("datas/preprocessing_kbo.csv")

In [41]:
climate = climate.drop(['datename'], axis=1)
climate = climate.reset_index(drop=True)
climate['weekend'] = climate['weekend'].fillna(0)

In [42]:
kbo.head(5)

,dates,parks,away,home,homewin,etcs,times
0,2017/03/14,대전,LG,한화,무,시범경기,13:00
1,2017/03/14,대구,KT,삼성,패,시범경기,13:00
2,2017/03/14,광주,두산,기아,승,시범경기,13:00
3,2017/03/14,사직,SK,롯데,패,시범경기,13:00
4,2017/03/14,마산,넥센,NC,무,시범경기,13:00


In [43]:
df = attendance.merge(climate, left_on="날짜", right_on="date")
df = df.drop(["date", "weekday", "행정구", "MSRSTE_NM"], axis=1)

In [44]:
df = df.merge(kbo, left_on=["날짜", "구장"], right_on=["dates", "parks"])
df = df.drop(["홈", "방문", "dates", "parks"], axis=1)

In [45]:
df.head(5)

,날짜,요일,구장,관중수,weekend,맑음,눈/비,min_c,max_c,NO2,O3,CO,SO2,PM10,PM25,away,home,homewin,etcs,times
0,2017/03/31,금,잠실,"21,121",0.0,0.0,1.0,5.0,12.0,0.038,0.021,0.6,0.004,47.0,27.0,한화,두산,승,정규시즌,19:00
1,2017/04/01,토,잠실,"20,636",1.0,0.0,1.0,5.0,15.0,0.022,0.033,0.5,0.004,22.0,14.0,한화,두산,패,정규시즌,14:00
2,2017/04/02,일,잠실,"22,457",1.0,1.0,0.0,3.0,16.0,0.025,0.037,0.5,0.005,50.0,30.0,한화,두산,승,정규시즌,14:00
3,2017/04/04,화,잠실,"16,411",0.0,1.0,0.0,7.0,22.0,0.049,0.033,0.7,0.007,78.0,37.0,삼성,LG,승,정규시즌,18:30
4,2017/04/06,목,잠실,"12,166",0.0,0.0,1.0,9.0,12.0,0.027,0.028,0.5,0.005,26.0,15.0,삼성,LG,승,정규시즌,18:30


In [46]:
df.columns

Index(['날짜', '요일', '구장', '관중수', 'weekend', '맑음', '눈/비', 'min_c', 'max_c',
       'NO2', 'O3', 'CO', 'SO2', 'PM10', 'PM25', 'away', 'home', 'homewin',
       'etcs', 'times'],
      dtype='object')

In [47]:
columns = ["Date", "WeekDay", "Park", "attendance", "Weekend", "Shiny", "Rain", "Min_c",
           "Max_c", "NO2", "O3", "CO", "SO2", "PM10", "PM25", "Away", "Home", "Result", "etc", "Time"]

In [48]:
df.columns = columns
df["attendance"] = df["attendance"].apply(
    lambda data: int(data.replace(",", "")))
df["Date"] = df["Date"].apply(pd.to_datetime)
df.set_index("Date", inplace=True)
df["year"] = df.index.year
df["month"] = df.index.month
df["quarter"] = df.index.quarter
df["Weekend"] = df["Weekend"].astype(int).astype(str)

In [49]:
df = df[["year", "month", "quarter", "WeekDay",  "Weekend", "Home", "Result", "Away",  "etc", "Time", "Park",  "Shiny", "Rain", "Min_c",
         "Max_c", "NO2", "O3", "CO", "SO2", "PM10", "PM25", "attendance"]]

In [50]:
df.to_csv("datas/master_table.csv", index=False)